In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## ChatGPTとGoogle検索の連携
- ChatGPTなどのLLMは過去のデータで学習しているため、最新の気象情報を保有していない。
- Google検索を使って気象情報を取得し、ChatGPTと連携させる

In [2]:
from langchain import SerpAPIWrapper, LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, Tool, initialize_agent, AgentType

llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")

tool_names = ["serpapi"]

tools = load_tools(tool_names, llm=llm)

weather_question = "今日の東京と北海道の天気を調べて、東京から北海道の出張で着ていくべき服装を考えてください"

### Zero-shot ReAct
ReActフレームワークを使用して、ツールの説明のみに基づいて使用するツールを決定するAgent。最も汎用的。

In [3]:
zero_shot_agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
zero_shot_agent.run(weather_question)



> Entering new AgentExecutor chain...
I should search for the weather in Tokyo and Hokkaido and then consider the appropriate clothing for a business trip from Tokyo to Hokkaido.
Action: Search
Action Input: "weather in Tokyo and Hokkaido today"
Observation: ['10 Day Weather-Sapporo, Hokkaido, Japan. As of 9:32 am JST. Today. 75°/63°. 12%. Wed 27 | Day. 75°. 12%. NE 6 mph. Mostly sunny. High near 75F.']
Thought:I need to search for the weather in Tokyo as well.
Action: Search
Action Input: "weather in Tokyo today"
Observation: {'type': 'weather_result', 'temperature': '86', 'unit': 'Fahrenheit', 'precipitation': '30%', 'humidity': '70%', 'wind': '4 mph', 'location': 'Tokyo, Japan', 'date': 'Wednesday', 'weather': 'Cloudy'}
Thought:I now know the final answer
Final Answer: The weather in Tokyo today is cloudy with a temperature of 86°F and a 30% chance of precipitation. The weather in Hokkaido today is mostly sunny with a high near 75°F. Based on this information, appropriate clothing

'The weather in Tokyo today is cloudy with a temperature of 86°F and a 30% chance of precipitation. The weather in Hokkaido today is mostly sunny with a high near 75°F. Based on this information, appropriate clothing for a business trip from Tokyo to Hokkaido would include a mix of light layers such as a shirt, blazer, and trousers, as well as a light jacket or sweater for cooler temperatures in the morning or evening.'

### OpenAI Functions
OpenAIのFunction Callingを利用するAgent。Zero-shot ReActと同様に、プログラム実行者の環境でfunctionを実行する。

In [4]:
openai_functions_agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
openai_functions_agent.run(weather_question)



> Entering new AgentExecutor chain...

Invoking: `Search` with `{'query': '今日の東京の天気'}`


{'type': 'weather_result', 'temperature': '86', 'unit': 'Fahrenheit', 'precipitation': '30%', 'humidity': '70%', 'wind': '4 mph', 'location': '日本、東京都', 'date': 'Wednesday', 'weather': 'Cloudy'}
Invoking: `Search` with `{'query': '今日の北海道の天気'}`


{'type': 'weather_result', 'temperature': '77', 'unit': 'Fahrenheit', 'precipitation': '20%', 'humidity': '78%', 'wind': '9 mph', 'location': '日本、北海道', 'date': 'Wednesday', 'weather': 'Clear in the morning but cloudy in the evening'}今日の東京の天気は曇りで、気温は86°F、降水確率は30%、湿度は70%、風速は4 mphです。一方、北海道の天気は朝は晴れていますが、夜には曇りになる予報です。気温は77°F、降水確率は20%、湿度は78%、風速は9 mphです。

東京から北海道への出張の際の服装について考えますと、東京は曇りの天気ですので、軽めのジャケットやセーターなどの上着を持っていくと良いでしょう。北海道は朝晴れていますが、夜には曇りになる予報ですので、長袖のシャツや薄手のカーディガンなど、寒さ対策も考慮した服装がおすすめです。また、北海道は東京に比べて風速が高いため、風を防ぐための軽い防風ジャケットやストールなども持参することをおすすめします。

どうぞ安全な出張をお過ごしください。

> Finished chain.


'今日の東京の天気は曇りで、気温は86°F、降水確率は30%、湿度は70%、風速は4 mphです。一方、北海道の天気は朝は晴れていますが、夜には曇りになる予報です。気温は77°F、降水確率は20%、湿度は78%、風速は9 mphです。\n\n東京から北海道への出張の際の服装について考えますと、東京は曇りの天気ですので、軽めのジャケットやセーターなどの上着を持っていくと良いでしょう。北海道は朝晴れていますが、夜には曇りになる予報ですので、長袖のシャツや薄手のカーディガンなど、寒さ対策も考慮した服装がおすすめです。また、北海道は東京に比べて風速が高いため、風を防ぐための軽い防風ジャケットやストールなども持参することをおすすめします。\n\nどうぞ安全な出張をお過ごしください。'

どんな関数がChatGPTに渡されているかは `functions` メソッドで確認できる。

In [5]:
openai_functions_agent.agent.functions

[{'name': 'Search',
  'description': 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}}]

## 自分で作った関数とChatGPTの連携

### Toolを `Tool.from_function` で定義
実行したい関数とその説明をもとにLangChain Agentで使えるToolを作成。

In [6]:
def recommend_item(date):
    print(date)
    return "Apple"

recommend_item_tool = Tool.from_function(
    func=recommend_item,
    name="RecommendItem",
    description="おすすめの商品を教えてくれます。"
)
tools = [recommend_item_tool]

In [7]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("おすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `RecommendItem` with `おしゃれなバッグ`


おしゃれなバッグ
Appleおしゃれなバッグのおすすめ商品はAppleです。

> Finished chain.


'おしゃれなバッグのおすすめ商品はAppleです。'

Invoking: \`RecommendItem\` with \`****\` の `****`　が、関数の引数として渡されるものとなっており、全く意味のないものを渡していることがわかる。

In [8]:
agent.agent.functions

[{'name': 'RecommendItem',
  'description': 'おすすめの商品を教えてくれます。',
  'parameters': {'properties': {'__arg1': {'title': '__arg1',
     'type': 'string'}},
   'required': ['__arg1'],
   'type': 'object'}}]

In [10]:
from pydantic.v1 import BaseModel, Field  # v1を指定しないとpydantic v2を使って動かなくなるので注意
from langchain.tools import tool

class RecommendItemInput(BaseModel):
    date: str = Field(description="おすすめしたい商品を絞るために使われる日付")

recommend_item_tool = Tool.from_function(
    func=recommend_item,
    name="RecommendItem",
    description="おすすめの商品を教えてくれます。",
    args_schema=RecommendItemInput
)
tools = [recommend_item_tool]

In [11]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("おすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `RecommendItem` with `{'date': '2022-01-10'}`


2022-01-10
Apple2022年1月10日のおすすめ商品はAppleです。

> Finished chain.


'2022年1月10日のおすすめ商品はAppleです。'

In [12]:
agent.agent.functions

[{'name': 'RecommendItem',
  'description': 'おすすめの商品を教えてくれます。',
  'parameters': {'title': 'RecommendItemInput',
   'type': 'object',
   'properties': {'date': {'title': 'Date',
     'description': 'おすすめしたい商品を絞るために使われる日付',
     'type': 'string'}},
   'required': ['date']}}]

### `tool` デコレータを使った実装
デコレータの引数にツールの名前を渡せる。toolの説明はdocstringに書く。

In [13]:
from langchain.tools import tool

@tool("RecommendItem")
def recommend_item_tool(date):
    """おすすめの商品を教えてくれます。"""
    print(date)
    return "Apple"

tools = [recommend_item_tool]

In [14]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("おすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `RecommendItem` with `{'date': '2021-10-15'}`


2021-10-15
Appleおすすめの商品は、Appleです。

> Finished chain.


'おすすめの商品は、Appleです。'

デコレータを使った実装では、引数としてdateが渡されていることがわかる。ただし、渡されている日付は適当。

In [15]:
agent.agent.functions

[{'name': 'RecommendItem',
  'description': 'RecommendItem(date) - おすすめの商品を教えてくれます。',
  'parameters': {'title': 'RecommendItemSchemaSchema',
   'type': 'object',
   'properties': {'date': {'title': 'Date'}},
   'required': ['date']}}]

デコレータのargs_schema引数にPydanticを渡して、引数の情報をChatGPTに連携できる。

In [16]:
@tool("RecommendItem", args_schema=RecommendItemInput)
def recommend_item_tool(date):
    """おすすめの商品を教えてくれます。"""
    print(date)
    return "Apple"

tools = [recommend_item_tool]

In [17]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("おすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `RecommendItem` with `{'date': '2022-10-01'}`


2022-10-01
Apple2022年10月1日のおすすめ商品はAppleです。

> Finished chain.


'2022年10月1日のおすすめ商品はAppleです。'

In [18]:
agent.run("昨日のおすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `RecommendItem` with `{'date': '2022-12-11'}`


2022-12-11
Apple昨日のおすすめ商品はAppleです。

> Finished chain.


'昨日のおすすめ商品はAppleです。'

このように今日や昨日などを与えたとしても、現在のバージョンでは正しい日付が返ってこない。

#### 正しい日付を取得するためのToolを追加

In [19]:
import datetime

@tool("GetDate")
def get_date_tool(n):
    """今日や昨日など日付を取得するときに使います。引数には今日を起点として何日前かをnで与えます。"""  # pydanticで引数の説明をしたほうがベター
    return str(datetime.date.today() - datetime.timedelta(days=n))

tools = [recommend_item_tool, get_date_tool]

In [20]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("今日のおすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 0}`


2023-09-27
Invoking: `RecommendItem` with `{'date': '2023-09-27'}`


2023-09-27
Apple今日のおすすめ商品は「Apple」です。

> Finished chain.


'今日のおすすめ商品は「Apple」です。'

In [21]:
agent.agent.functions

[{'name': 'RecommendItem',
  'description': 'RecommendItem(date) - おすすめの商品を教えてくれます。',
  'parameters': {'title': 'RecommendItemInput',
   'type': 'object',
   'properties': {'date': {'title': 'Date',
     'description': 'おすすめしたい商品を絞るために使われる日付',
     'type': 'string'}},
   'required': ['date']}},
 {'name': 'GetDate',
  'description': 'GetDate(n) - 今日や昨日など日付を取得するときに使います。引数には今日を起点として何日前かをnで与えます。',
  'parameters': {'title': 'GetDateSchemaSchema',
   'type': 'object',
   'properties': {'n': {'title': 'N'}},
   'required': ['n']}}]

In [22]:
@tool("RecommendItem", args_schema=RecommendItemInput)
def recommend_item_tool(date):
    """与えられた日付の自社のおすすめ商品を教えてくれます。"""
    if int(date[-2:]) % 2 == 0:
        return "Apple"
    else:
        return "Orange"
tools = [recommend_item_tool, get_date_tool]

In [23]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("今日のおすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 0}`


2023-09-27
Invoking: `RecommendItem` with `{'date': '2023-09-27'}`


Orange今日のおすすめ商品は「オレンジ」です。

> Finished chain.


'今日のおすすめ商品は「オレンジ」です。'

In [24]:
agent.run("一昨日のおすすめ商品を教えてください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 2}`


2023-09-25
Invoking: `RecommendItem` with `{'date': '2023-09-25'}`


Orange一昨日のおすすめ商品は「Orange」です。

> Finished chain.


'一昨日のおすすめ商品は「Orange」です。'

In [25]:
@tool("ItemPrice")
def item_price_tool(item_name):
    """商品名を受け取り、商品の値段を返します"""
    if item_name == "Apple":
        return 100
    elif item_name == "Orange":
        return 200

tools = [recommend_item_tool, get_date_tool, item_price_tool]

In [26]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("昨日のおすすめ商品の値段を教えてください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 1}`


2023-09-26
Invoking: `RecommendItem` with `{'date': '2023-09-26'}`


Apple
Invoking: `ItemPrice` with `{'item_name': 'Apple'}`


100昨日のおすすめ商品「Apple」の値段は100円です。

> Finished chain.


'昨日のおすすめ商品「Apple」の値段は100円です。'

In [28]:
agent.run("今日のおすすめ商品の値段を教えてください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 0}`


2023-09-27
Invoking: `RecommendItem` with `{'date': '2023-09-27'}`


Orange
Invoking: `ItemPrice` with `{'item_name': 'Orange'}`


200今日のおすすめ商品はオレンジで、価格は200円です。

> Finished chain.


'今日のおすすめ商品はオレンジで、価格は200円です。'

### 結果を外部出力する
今回は簡易的にテキストファイルに書き込んで観る。

In [29]:
!rm -f output.txt

In [30]:
@tool("SaveData")
def save_data_tool(date, item_name, item_price):
    """日付と商品名と商品の値段を受け取り、データベースに書き込みます"""
    with open("output.txt", "a") as f:
        f.write(f"{date},{item_name},{item_price}")

tools = [recommend_item_tool, get_date_tool, item_price_tool, save_data_tool]

In [31]:
agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
agent.run("今日のおすすめ商品の値段を調べてデータベースに書き込んでください")



> Entering new AgentExecutor chain...

Invoking: `GetDate` with `{'n': 0}`


2023-09-27
Invoking: `RecommendItem` with `{'date': '2023-09-27'}`


Orange
Invoking: `ItemPrice` with `{'item_name': 'Orange'}`


200
Invoking: `SaveData` with `{'date': '2023-09-27', 'item_name': 'Orange', 'item_price': 200}`


None今日のおすすめ商品は「Orange」で、価格は「200円」です。データベースに書き込みました。

> Finished chain.


'今日のおすすめ商品は「Orange」で、価格は「200円」です。データベースに書き込みました。'

In [32]:
!cat output.txt

2023-09-27,Orange,200